# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,276.81,91,99,16.27,TF,1694713520
1,1,blackmans bay,-43.0167,147.3167,288.22,62,100,3.89,AU,1694713517
2,2,chinde,26.8418,88.0763,292.02,91,45,1.84,NP,1694714114
3,3,new norfolk,-42.7826,147.0587,288.87,68,100,1.11,AU,1694713568
4,4,aasiaat,68.7098,-52.8699,274.75,92,95,2.37,GL,1694713543


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    color="City",
    size="Humidity",
    alpha=0.5)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
weather_df = ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values

weather_df = weather_df.dropna()

# Display sample data
weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp Celsius
119,119,shakawe,-18.3667,21.8500,24.08,19,0,2.66,BW,1694714135,24.08
212,212,portland,45.5234,-122.6762,23.88,66,0,3.09,US,1694713565,23.88
244,244,maghar,32.8898,35.4070,25.96,73,0,2.17,IL,1694714152,25.96
245,245,kailua-kona,19.6406,-155.9956,26.95,72,0,2.57,US,1694714012,26.95
278,278,hotan,37.0997,79.9269,24.47,17,0,2.40,CN,1694714157,24.47


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
119,shakawe,BW,-18.3667,21.8500,19,
212,portland,US,45.5234,-122.6762,66,
244,maghar,IL,32.8898,35.4070,73,
245,kailua-kona,US,19.6406,-155.9956,72,
278,hotan,CN,37.0997,79.9269,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [66]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
     "apiKey": "a01b640a1f534747a7a7c00a6e084ef9",
    "limit": 1,
    "radius": radius,
    "categories": "accommodation"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lon = row["Lat"], row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
shakawe - nearest hotel: Hawk Guest House
portland - nearest hotel: the Hoxton
maghar - nearest hotel: kibbutz inbar country lodging
kailua-kona - nearest hotel: Kona Seaside Hotel
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
eureka - nearest hotel: Eureka Inn, Trademark Collection by Wyndham
saint-pierre - nearest hotel: Tropic Hotel
opuwo - nearest hotel: ANNA FLAT
baker city - nearest hotel: Knights Inn
ilheus - nearest hotel: No hotel found
beira - nearest hotel: Pousada Moderna
kaoma - nearest hotel: Phiri’s Guest House
tejen - nearest hotel: Tejen Myhmanhanasy
alghero - nearest hotel: Actinia B&B
batavia - nearest hotel: Comfort Inn & Suites Geneva


,City,Country,Lat,Lng,Humidity,Hotel Name
119,shakawe,BW,-18.3667,21.8500,19,Hawk Guest House
212,portland,US,45.5234,-122.6762,66,the Hoxton
244,maghar,IL,32.8898,35.4070,73,kibbutz inbar country lodging
245,kailua-kona,US,19.6406,-155.9956,72,Kona Seaside Hotel
278,hotan,CN,37.0997,79.9269,17,Hua Yi International Hotel (accepts foreigners)
282,eureka,US,40.8021,-124.1637,76,"Eureka Inn, Trademark Collection by Wyndham"
299,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
317,opuwo,NaN,-18.0607,13.8400,12,ANNA FLAT
340,baker city,US,44.7749,-117.8344,54,Knights Inn
372,ilheus,BR,-14.7889,-39.0494,83,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [70]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",  
    hover_cols=["City", "Hotel Name", "Country"],
    frame_width=800,
    frame_height=600,
    color="Humidity",  
    size=10,
    alpha=0.7,
    title="Hotels in Ideal Cities"
)

# Display the map
map_plot.opts(tools=['hover'], active_tools=['wheel_zoom'])

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)